* PROJEKCT CPM FILIP NIEWCZAS, WIKTORIA DOMAŃSKA, MARCEL PASSON *

Detecting texts generated by large language models
As large language models are increasingly being used to generate texts in the context of everyday communication, blog posts, advertisements etc., there is a practical problem of distinguishing between artificially generated texts and those authored by humans. It is also interesting to investigate if there are any systematic differences between such texts on the language level.
Some datasets of real vs generated texts:
https://github.com/jmpu/DeepfakeTextDetection
https://github.com/liamdugan/human-detection/
https://huggingface.co/datasets/aadityaubhat/GPT-wiki-intro
https://github.com/ICTMCG/Awesome-Machine-Generated-Text (long list under Datasets section)
Choose a dataset from a particular usage domain. Formulate some initial hypotheses.
Perform an extensive exploration of the dataset. What features are characteristic for the generated texts?
Try to build your own classifier detecting generated content.
Complement your work with a qualitative analysis of generated texts.


In [1]:
%pip install nlp

Note: you may need to restart the kernel to use updated packages.


In [2]:
from nlp import Dataset
import numpy as np
from transformers import AutoTokenizer, pipeline, Trainer, TrainingArguments, AutoModelForSequenceClassification,DataCollatorWithPadding
import pandas as pd

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-11 13:09:02.412903: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:


# Load the .jsonl file
file_path = '/Users/filip/Desktop/Cognitive/cpm/project /RedditBot.jsonl' #'/content/drive/MyDrive/CogSCi/CPM II /Projekt CPM /project /RedditBot.jsonl'
data = pd.read_json(file_path, lines=True)

# Display the first few rows
print("First few rows of the dataset:")
print(data.head())

# Display summary information
print("\nSummary information of the dataset:")
print(data.info())

# Display basic statistics
print("\nBasic statistics of the dataset:")
print(data.describe())

# Check for missing values
missing_values = data.isnull().sum()
print("\nMissing values in each column:")
print(missing_values)

# Calculate the length of each text
data['text_length'] = data['text'].apply(len)

First few rows of the dataset:
   id                                               text    label
0   0  To Kill a Mockingbird  is an excellent example...  machine
1   0  there must be a reason to make your claim...so...    human
2   0   God is an idea that humans believe in, and th...  machine
3   0  If you need to make menial philosophical point...    human
4   0   Probably one of the worst inventions is eugen...  machine

Summary information of the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1774 entries, 0 to 1773
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1774 non-null   int64 
 1   text    1774 non-null   object
 2   label   1774 non-null   object
dtypes: int64(1), object(2)
memory usage: 41.7+ KB
None

Basic statistics of the dataset:
           id
count  1774.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0

Missing values in each column:
id

In [5]:
#  transforming dataset - delete columns ID and text_lenght, drop duplicates, drop missing values
data = data.drop(columns=['id', 'text_length']).drop_duplicates().dropna()

In [6]:
data.head(5)

,text,label
0,To Kill a Mockingbird is an excellent example...,machine
1,there must be a reason to make your claim...so...,human
2,"God is an idea that humans believe in, and th...",machine
3,If you need to make menial philosophical point...,human
4,Probably one of the worst inventions is eugen...,machine


In [7]:
#Changing values machine-human to 0:1
data['label'] = data['label'].map({'machine': 0, 'human': 1})
data.sample(5)

,text,label
715,Okay this may seem stupid but here it goes: ce...,1
1057,My best friends. I drove up like 100 miles nor...,1
105,That's based on old information. Only some chi...,1
574,I don't want to use this as some sort of plug ...,1
25,\n\nI think that the biggest challenge in sca...,0


In [8]:
data['label'].mean() # fifty-fifty almost!

0.49745042492917846

In [9]:
dataset = Dataset.from_pandas(data)

In [10]:
#Parameters for the model
train_fraction = 0.7 # fraction of a dataset used for training (the rest used for validation)
num_train_epochs = 3 # epochs to train
batch_size = 4 # batch size for training and validation
warmup_steps = 50
weight_decay = 0.02


In [11]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased', use_fast=True,low_cpu_mem_usage=False)

In [12]:

def encode_examples(example):
    # Encode the text and return the encoding which includes 'input_ids' and 'attention_mask'
    return tokenizer(example['text'], truncation=True, padding='max_length')


dataset_encoded = dataset.map(encode_examples, batched=True)


print(dataset_encoded)

  0%|          | 0/2 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 2/2 [00:01<00:00,  1.05it/s]

Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None), '__index_level_0__': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 1765)


In [13]:
splitted_dataset = dataset_encoded.train_test_split(train_fraction) #0.7 for training, 0.3 for validation

100%|██████████| 2/2 [00:01<00:00,  1.35it/s]


In [14]:
splitted_dataset

{'train': Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None), '__index_level_0__': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 529),
 'test': Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None), '__index_level_0__': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 1236)}

In [15]:
# DataCollatorWithPadding creates batch of data. It also dynamically pads text to the
#  length of the longest element in the batch, making them all the same length.
#  It's possible to pad your text in the tokenizer function with padding=True, dynamic padding is more efficient.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # Tokenizer - DistilBert

In [16]:
print(splitted_dataset['train'][0].keys())

dict_keys(['__index_level_0__', 'attention_mask', 'input_ids', 'label', 'text'])


In [17]:
tokenizer.decode(splitted_dataset['train'][0]['input_ids'])

"[CLS] I'd like to add to the discourse that stability is not directly linked to allergenicity in a mechanistic way that can be explained by immunologists. Peanut is also reported to interact with a receptor called DC - SIGN that enhances its uptake into the cells responsible for initiating immune responses. Additionally, western preparation of peanuts via dry roasting changes the structure of sugars in the peanut, adding things called advanced glycosylation end products, which are targeted by a specialized receptor, which is also known for uptake and activating the immune system. In general, its not clear why anyone is allergic to specific things. Some allergens ( like peanut, shellfish, house dust mite etc ) have protein cleaving and trypsin inhibiting properties that the immune system seems to react to. Others like milk dont have those properties. Its all still quite a mystery and is very very hard to study because by the time a patient comes to the clinic, they are already fully al

In [18]:
# Loading the model

BERT_MODEL = 'distilbert-base-cased'

model = AutoModelForSequenceClassification.from_pretrained(
    BERT_MODEL, num_labels=2,
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False # Whether the model returns all hidden-states.
)

model.config.id2label = {0: 'ROBOT', 1: 'HUMAN'}

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# number of trainable parameters
print(model.num_parameters(only_trainable=True)/1e6)

65.783042


In [20]:
%pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [21]:
from datasets import load_metric

metric = load_metric("accuracy") # do oceny

def compute_metrics(eval_pred):
    logits, labels = eval_pred #rozdzielenie przewidywania (logits) i etykiet (labels)
    predictions = np.argmax(logits, axis=-1)  #argmax zwraca indeksy największych wartości w kolumnie wzdłuż osi
    return metric.compute(predictions=predictions, references=labels)

/var/folders/np/t6zn6slj5yx_33m6c396sq0m0000gq/T/ipykernel_49710/1483163414.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy") # do oceny
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [22]:
%pip install transformers[torch]
%pip install mlflow


zsh:1: no matches found: transformers[torch]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [23]:
import accelerate
print(accelerate.__version__)

0.30.1


In [24]:
def is_accelerate_available():
    return True
training_args = TrainingArguments(
    output_dir="./project cpm2 ",
    logging_dir='./logs',
    num_train_epochs=num_train_epochs, # 3
    per_device_train_batch_size=2, #16 - number of samples to process at once per batch
    per_device_eval_batch_size=2, #16
    logging_strategy='steps', # log every step
    logging_first_step=True,
    load_best_model_at_end=True, #trainer will load the best model found during training at the end of training
    logging_steps=1,
    evaluation_strategy='epoch',# when evaluate model - after each epoch
    warmup_steps=warmup_steps, #50
    weight_decay=weight_decay, #0.02
    eval_steps=1,
    save_strategy='epoch',
    report_to="mlflow",  # log to mlflow
)



# Define the trainer:
# instantiate the trainer class and check for available devices
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=splitted_dataset['train'],
    eval_dataset=splitted_dataset['test'],
    data_collator=data_collator # A function to batch together samples of data.
)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [25]:
trainer.evaluate()

100%|██████████| 618/618 [11:04<00:00,  1.10s/it]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALL

{'eval_loss': 0.6935293078422546,
 'eval_accuracy': 0.5032362459546925,
 'eval_runtime': 666.2794,
 'eval_samples_per_second': 1.855,
 'eval_steps_per_second': 0.928}

In [26]:
trainer.train()

  0%|          | 1/795 [00:04<52:36,  3.98s/it]

{'loss': 0.6845, 'grad_norm': 6.480278968811035, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}


  0%|          | 2/795 [00:07<48:15,  3.65s/it]

{'loss': 0.6951, 'grad_norm': 2.9086215496063232, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}


  0%|          | 3/795 [00:10<43:15,  3.28s/it]

...

                                                 
100%|██████████| 795/795 [1:08:57<00:00,  2.33s/it]

{'eval_loss': 0.18705609440803528, 'eval_accuracy': 0.9676375404530745, 'eval_runtime': 631.4271, 'eval_samples_per_second': 1.957, 'eval_steps_per_second': 0.979, 'epoch': 3.0}


100%|██████████| 795/795 [1:09:01<00:00,  5.21s/it]

{'train_runtime': 4141.4021, 'train_samples_per_second': 0.383, 'train_steps_per_second': 0.192, 'train_loss': 0.1910579867478231, 'epoch': 3.0}


TrainOutput(global_step=795, training_loss=0.1910579867478231, metrics={'train_runtime': 4141.4021, 'train_samples_per_second': 0.383, 'train_steps_per_second': 0.192, 'train_loss': 0.1910579867478231, 'epoch': 3.0})

In [27]:
trainer.evaluate()

100%|██████████| 618/618 [10:28<00:00,  1.02s/it]


{'eval_loss': 0.18705609440803528,
 'eval_accuracy': 0.9676375404530745,
 'eval_runtime': 629.8774,
 'eval_samples_per_second': 1.962,
 'eval_steps_per_second': 0.981,
 'epoch': 3.0}

In [28]:
trainer.save_model()

In [29]:
tokenizer.save_vocabulary(save_directory="./project cpm2 ")

('./project cpm2 /vocab.txt',)

In [30]:
#from google.colab import drive
#drive.mount('/content/drive')

In [34]:
%pip install tf-keras

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [36]:
pipe = pipeline("text-classification", "/Users/filip/Desktop/Cognitive/cpm/project cpm2 ", tokenizer=BERT_MODEL)



sample_text = '''
I'm human and I'm proud of it!
'''


pipe(sample_text, top_k=None)

[{'label': 'HUMAN', 'score': 0.9989688396453857},
 {'label': 'ROBOT', 'score': 0.00103111588396132}]